# A Testing Framework: Exercises

## Looping Over `globals`

**What happens if you run:**

In [1]:
for name in globals():
    print(name)

__name__


RuntimeError: dictionary changed size during iteration

**What happens if you run:**

In [2]:
name = None
for name in globals():
    print(name)

__name__
__doc__
__package__
__loader__
__spec__
__builtin__
__builtins__
_ih
_oh
_dh
In
Out
get_ipython
exit
quit
_
__
___
_i
_ii
_iii
_i1
name
_i2


**Why?**

The first run results in an error because the `globals()` dictionary changes size during the course of the for loop. This is because when the variable `name` is set to the first key in `globals()` (i.e. the first iteration of the for loop), the variable `name` is *added* to the`globals()` dictionary!

## Counting Results

1.  **Modify the test framework so that it reports which tests passed, failed, or had errors and also reports a summary of how many tests produced each result.**

In [3]:
def run_tests(prefix):
    all_names = [n for n in globals() if n.startswith(prefix)]
    results = {"pass": 0, "fail": 0, "error": 0, "skip": 0, "expected failure": 0}
    for name in all_names:
        func = globals()[name]
        kind = classify(func)
        try:
            if kind == "skip":
                print(f"skip: {name}")
                results["skip"] += 1
            else:
                func()
                print(f"pass: {name}")
                results["pass"] += 1
        except AssertionError as e:
            if kind == "fail":
                print(f"pass (expected failure): {name}")
                results["expected failure"] += 1
            else:
                print(f"fail: {name} {str(e)}")
                results["fail"] += 1
        except Exception as e:
            print(f"error: {name} {str(e)}")
            results["error"] += 1
            
    print("\n\nTesting Summary:")
    print(f"pass {results['pass']}")
    print(f"fail {results['fail']}")
    print(f"error {results['error']}")
    print(f"skip {results['skip']}")
    print(f"expected failure {results['expected failure']}")

2.  **Write unit tests to check that your answer to part 1 works correctly.**

In [4]:
def sign(value):
    if value < 0:
        return -1
    else:
        return 1

def test_sign_negative():
    assert sign(-3) == -1

test_sign_negative.skip = True

def test_sign_positive():
    assert sign(19) == 1
    
def test_sign_zero():
    assert sign(0) == 0
    
test_sign_zero.fail = True

def test_sign_zero_again():
    assert sign(0) == 0

def test_sign_error():
    assert sgn(1) == 1
    
def classify(func):
    if hasattr(func, "skip") and func.skip:
        return "skip"
    if hasattr(func, "fail") and func.fail:
        return "fail"
    return "run"

In [5]:
run_tests('test_')

skip: test_sign_negative
pass: test_sign_positive
pass (expected failure): test_sign_zero
fail: test_sign_zero_again 
error: test_sign_error name 'sgn' is not defined


Testing Summary:
pass 1
fail 1
error 1
skip 1
expected failure 1


3.  **Think of another plausible way to interpret part 1 that *wouldn't* pass the tests you wrote for part 2.**

    Here we're trying to break the framework

## Failing on Purpose

**Putting assertions into code to check that it is behaving correctly is called defensive programming. It's a good practice, but we should make sure those assertions are failing when they're supposed to, just as we should test our smoke detectors every once in a while.**

**Modify the tester so that if a test function's docstring is `"test:assert"`, the test passes if it raises an `AssertionError` and fails if it does not. Tests whose docstring don't contain `"test:assert"` should behave as before.**

------------------------------------------------------------------------

In [6]:
def run_tests(prefix):
    all_names = [n for n in globals() if n.startswith(prefix)]
    results = {"pass": 0, "fail": 0, "error": 0, "skip": 0, "expected failure": 0}
    for name in all_names:
        func = globals()[name]
        kind = classify(func)
        try:
            if kind == "skip":
                print(f"skip: {name}")
                results["skip"] += 1
            else:
                func()
                if func.__doc__ == "test:assert":
                  print(f"fail: {name}")
                  results["fail"] += 1
                else:
                  print(f"pass: {name}")
                  results["pass"] += 1
        except AssertionError as e:
            if func.__doc__ == "test:assert":
                print(f"pass: {name}")
                results["pass"] += 1
            else:
                print(f"fail: {name} {str(e)}")
                results["fail"] += 1
        except Exception as e:
            print(f"error: {name} {str(e)}")
            results["error"] += 1
            
    print("\n\nTesting Summary:")
    print(f"pass {results['pass']}")
    print(f"fail {results['fail']}")
    print(f"error {results['error']}")
    print(f"skip {results['skip']}")
    print(f"expected failure {results['expected failure']}")

In [7]:
# should raise AssertionError: expect to pass
def test2_sign_zero():
    """test:assert"""
    assert sign(0) == 0
    
# should not raise AssertionError: expect to fail
def test2_sign_positive():
    """test:assert"""
    assert sign(1) == 1
    
run_tests("test2_")

pass: test2_sign_zero
fail: test2_sign_positive


Testing Summary:
pass 1
fail 1
error 0
skip 0
expected failure 0


## Setup and Teardown

**Testing frameworks often allow programmers to specify a `setup` function that is to be run before each test and a corresponding `teardown` function that is to be run after each test. (`setup` usually re-creates complicated test fixtures, while `teardown` functions are sometimes needed to clean up after tests, e.g., to close database connections or delete temporary files.)**

**Modify the testing tool in this chapter so that if a file of tests contains a function called `setup` then the tool calls it exactly once before running each test in the file. Add a similar way to register a `teardown` function.**

Greg said that dealing with loading files from the string names is hard. Instead, I'll continue searching global functions. This testing framework assumes that the `setup` and `teardown` functions would also be found in `globals()` if they did exist.

In [8]:
def run_tests(prefix):
    all_globals = globals()
    all_names = [n for n in all_globals if n.startswith(prefix)]
    if 'setup' in all_globals:
      print('setting up...\n')
      setup()
    else:
      print('no setup\n')
    
    results = {"pass": 0, "fail": 0, "error": 0, "skip": 0, "expected failure": 0}
    for name in all_names:
        func = globals()[name]
        kind = classify(func)
        try:
            if kind == "skip":
                print(f"skip: {name}")
                results["skip"] += 1
            else:
                func()
                if func.__doc__ == "test:assert":
                  print(f"fail: {name}")
                  results["fail"] += 1
                else:
                  print(f"pass: {name}")
                  results["pass"] += 1
        except AssertionError as e:
            if func.__doc__ == "test:assert":
                print(f"pass: {name}")
                results["pass"] += 1
            else:
                print(f"fail: {name} {str(e)}")
                results["fail"] += 1
        except Exception as e:
            print(f"error: {name} {str(e)}")
            results["error"] += 1
    
    if 'teardown' in all_globals:
      print('\ntearing down...')
      teardown()
    else:
      print('\nno teardown')
            
    print("\n\nTesting Summary:")
    print(f"pass {results['pass']}")
    print(f"fail {results['fail']}")
    print(f"error {results['error']}")
    print(f"skip {results['skip']}")
    print(f"expected failure {results['expected failure']}")

In [9]:
# should raise AssertionError: expect to pass
def test3_sign_zero():
    """test:assert"""
    assert sign(0) == 0
    
def test3_sign_positive():
    for pos in [1,2,3]:
      assert sign(1) == 1

run_tests("test3_")

no setup

pass: test3_sign_zero
pass: test3_sign_positive

no teardown


Testing Summary:
pass 2
fail 0
error 0
skip 0
expected failure 0


Suppose that the list `[1,2,3]` is some hard-to-load dataset or object that we want to use throughout multiple tests. We don't want to reload it for each test, so we can define it globally as part of the setup. But it is only useful in the context of testing, so we want to delete it globally in the teardown.

In [10]:
def setup():
    global test_pos_list 
    test_pos_list = [1,2,3]

# should raise AssertionError: expect to pass
def test3_sign_zero():
    """test:assert"""
    assert sign(0) == 0
    
def test3_sign_positive():
    for pos in test_pos_list:
      assert sign(1) == 1
      
def teardown():
    global test_pos_list
    del test_pos_list
    
run_tests("test3_")

# check that teardown worked correctly
assert 'test_pos_list' not in globals()

setting up...

pass: test3_sign_zero
pass: test3_sign_positive

tearing down...


Testing Summary:
pass 2
fail 0
error 0
skip 0
expected failure 0
